# Vehicle Detection Project
The goals / steps of this project are the following:
- Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
    - Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.
    - Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
- Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
- Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
- Estimate a bounding box for vehicles detected.

# Imports

In [ ]:
import cv2
import random
import glob
import time
import pickle
import numpy as np
import matplotlib
matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from skimage.feature import hog
from scipy.ndimage.measurements import label
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Parameters

In [7]:
### TODO: Tweak these parameters and see how the results change.
color_space = 'YUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
bins_range = (0, 256) # Range of histogram bins, should be (0,1) for .png and (0,256) for .jpg
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()
max_center_offset = 100 # Allowable search range for a new vehicle
max_frame_hist = 7 # Number of frames to average the vehicle history over
max_loss_count = 3 # Number of allowed missing frames before the vehicle is considered lost
min_lifetime = 3 # Number of consecutive frames needed to see vehicle
# Search windows and scale
ystart = [400, 400, 400, 400]
ystop = [464, 496, 512, 628]
scale = [1.0, 1.5, 1.75, 2.0]



# Vehicle Detection Functions

In [3]:
def convert_color(img, color_space):
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        feature_image = np.copy(img)
    return feature_image


def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        feature_image = convert_color(image, color_space)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features
    
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, color_space, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    bboxes = []

    # Scale image to 0-1
    if np.amax(img) > 1:
        img = img.astype(np.float32)/255
    
    # Mask search area and convert color space 
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = convert_color(img_tosearch, color_space)

    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                bboxes.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)))
                
    return bboxes

def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def find_heatmap(image):
    box_list = []
    # Search through the image for cars
    for search in zip(ystart, ystop, scale):
        bboxes = find_cars(image, search[0], search[1], search[2], svc, X_scaler, color_space, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
        box_list += bboxes
        
    # Read in image similar to one shown above 
    heat = np.zeros_like(image[:,:,0]).astype(np.float)

    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    car_IDs = np.copy(range(0, labels[1]))

    return heatmap, labels, car_IDs
    
def convert_heatmap_to_bbox(img,labels,car_IDs):
    bbox = []
    bbox_center = []
    # Iterate through all detected cars
    for car_number in car_IDs:
        # Find pixels with each car_number label value
        nonzero = (labels[0] == (car_number+1)).nonzero()
        
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        # Define bounding boxes based on min/max x and y
        bbox.append(((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy))))
        bbox_center.append(((np.max(nonzerox)+np.min(nonzerox))/2,(np.max(nonzeroy) + np.min(nonzeroy))/2))
    
    return bbox, bbox_center

def find_existing_targets(bbox, bbox_center, car_IDs):
    # Find the most recent bounding box for each existing target
    for target in targets:
        offset = np.zeros_like(car_IDs)
        # Check if a target already exists
        if target.lifetime > 0:
            # Find the distance from each bounding box to the vehicle
            for car_index, car_number in enumerate(car_IDs):
                if car_number != 99:
                    offset[car_index] = np.linalg.norm(np.asarray(target.center) - np.asarray(bbox_center[car_index]))
                else:
                    offset[car_index] = max_center_offset

            # If the closest box is within bounds, then a new box has been found this frame for this vehicle
            if (np.amin(offset) < max_center_offset):
                target.detected = True
                target.lifetime += 1
                target.LossCount = 0
                target.ID = car_IDs[np.argmin(offset)]
                
                # If the running average is full, delete the last entry
                if target.lifetime > max_frame_hist:
                    del target.rec_bbox[0]

                # Append the newest entry and calculate the new average position
                target_bbox = np.asarray(bbox[np.argmin(offset)])
                target.rec_bbox.append(target_bbox)
                target.avg_bbox = (np.sum(np.stack(target.rec_bbox), axis = 0)/len(target.rec_bbox)).astype(int)
                target.center = ((target.avg_bbox[0][0]+target.avg_bbox[1][0])/2,(target.avg_bbox[0][1]+target.avg_bbox[1][1])/2)
            
                car_IDs[np.argmin(offset)] = 99 # Set the ID to 99 to indicate it has been used this frame
            
    return car_IDs
    
def find_new_targets(bbox,bbox_center,car_IDs):
    # Find bounding boxes for new targets
    for target in targets:
        if target.lifetime == 0:
            #Start at the end of the list, which is the bottom of the frame
            for car_number in car_IDs[::-1]:

                #fix this next line, skipping lots of cars by only checking first labels
                if car_number != 99:
                    if target.lifetime == 0:
                        target.detected = True
                        target.lifetime += 1
                        target.LossCount = 0
                        target.ID = car_number
                        target_bbox = np.asarray(bbox[car_number])
                        target.rec_bbox = [target_bbox]
                        target.avg_bbox = target_bbox
                        target.center = ((target.avg_bbox[0][0]+target.avg_bbox[1][0])/2,(target.avg_bbox[0][1]+target.avg_bbox[1][1])/2)

                        car_IDs[car_number] = 99 # Set the ID to 99 to indicate it has been used this frame
    
def draw_labeled_bboxes(img):
    # Draw the bounding boxes for identified vehicles
    for target in targets:
        # If a car was not found, increment the loss counter
        if not target.detected:
            target.LossCount += 1
            # If the target hasn't been seen within the max allowed frames, delete the target history
            if target.LossCount > max_loss_count:
                target.reset()                
        if (target.lifetime >= min_lifetime):# and (target.ID is not None):
            # Draw the box on the image
            cv2.rectangle(img, (target.avg_bbox[0][0],target.avg_bbox[0][1]),
                          (target.avg_bbox[1][0], target.avg_bbox[1][1]), (0,0,255), 6)

    # Return the image
    return img

def pipeline(image, show_heat = False):
    # Init variables for loop
    for target in targets:
        target.detected = False
        
    # Get the heatmap of the current frame
    heatmap, labels, car_IDs = find_heatmap(image)
    
    # Correlate identified targets to vehicles
    if len(car_IDs):
        bbox, bbox_center = convert_heatmap_to_bbox(image,labels,car_IDs)
        unused_car_IDs = find_existing_targets(bbox, bbox_center, car_IDs)
        find_new_targets(bbox, bbox_center, unused_car_IDs)

    # Draw the bounding boxes
    draw_img = draw_labeled_bboxes(np.copy(image))
    
    if show_heat:
        return heatmap, draw_img
    else:
        return draw_img
    
class Vehicle():
    def __init__(self):
        self.detected = False # Was the vehicle detected this frame
        self.ID = None # The ID of the vehicle (label number)
        self.LossCount = 0 # Number of frames since the vehicle was last seen
        self.lifetime = 0 # Number of frames that the vehicle has been seen
        self.avg_bbox = [(None, None),(None, None)] # Average position over the last N frames
        self.rec_bbox = [None] # History of last N positions
        self.center = [None,None] # Centroid of current vehicle
        
    def reset(self):
        self.detected = False # Was the vehicle detected this frame
        self.ID = None # The ID of the vehicle (label number)
        self.LossCount = 0 # Number of frames since the vehicle was last seen
        self.lifetime = 0 # Number of frames that the vehicle has been seen
        self.avg_bbox = [(None, None),(None, None)] # Average position over the last N frames
        self.rec_bbox = [None] # History of last N positions
        self.center = [None,None] # Centroid of current vehicle


# Test Pipeline on Images

In [22]:
svc_pickle = pickle.load( open("svc_pickle.pkl", "rb" ) )
svc = svc_pickle["svc"]
X_scaler = svc_pickle["X_scaler"]

# Create a list of vehicles for tracking
targets = [Vehicle(),Vehicle(),Vehicle()]
frame_count = 0
fnames = glob.glob("test_video_frames/*.jpg")
plt.close()
for fname in fnames:
    frame_count += 1
    print("######################################## FRAME {}".format(frame_count))
    image = mpimg.imread(fname)
    heatmap, draw_img = pipeline(image, show_heat = True)
    fig = plt.figure()
    plt.subplot(121)
    plt.imshow(draw_img)
    plt.title('Car Positions')
    plt.subplot(122)
    plt.imshow(heatmap, cmap='hot')
    plt.title('Heat Map')
    fig.tight_layout()
    plt.show()

######################################## FRAME 1
######################################## FRAME 2
######################################## FRAME 3
######################################## FRAME 4
######################################## FRAME 5
######################################## FRAME 6
######################################## FRAME 7
######################################## FRAME 8
######################################## FRAME 9
######################################## FRAME 10
######################################## FRAME 11
######################################## FRAME 12
######################################## FRAME 13
######################################## FRAME 14
######################################## FRAME 15
######################################## FRAME 16
######################################## FRAME 17
######################################## FRAME 18
######################################## FRAME 19
######################################## FRAME 20
#########

# Test Pipeline on Videos

In [8]:
svc_pickle = pickle.load( open("svc_pickle.pkl", "rb" ) )
svc = svc_pickle["svc"]
X_scaler = svc_pickle["X_scaler"]

# Create a list of vehicles for tracking
targets = [Vehicle(),Vehicle(),Vehicle()]

# Choose the video file
video = 'project_video'
#video = 'test_video'
length = '_full_length'
#length = '_clipped'

# Increment the video count
count = 0
file_list = glob.glob("output_videos/*.mp4")
for file in file_list:
    if video+length in file:
        count += 1

output = 'output_videos/{}{}_{}.mp4'.format(video, length, count)

if length == '_full_length':
    clip1 = VideoFileClip("{}.mp4".format(video))
if length == '_clipped':
    clip1 = VideoFileClip("{}.mp4".format(video)).subclip(4,7)
clip = clip1.fl_image(pipeline) #NOTE: this function expects color images!!

%time clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video output_videos/project_video_full_length_8.mp4
[MoviePy] Writing video output_videos/project_video_full_length_8.mp4


100%|█████████▉| 1260/1261 [21:29<00:01,  1.01s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video_full_length_8.mp4 

Wall time: 21min 31s


# Train the Classifier

In [23]:
# Read in cars and notcars
car_images = glob.glob('vehicles/*/*.png')
notcar_images = glob.glob('non-vehicles/*/*.png')
cars = []
notcars = []
for image in car_images:
    cars.append(image)
for image in notcar_images:
    notcars.append(image)

# Reduce the sample size
sample_size = 2000
random.shuffle(cars)
random.shuffle(notcars)
sample_cars = cars[0:sample_size]
sample_notcars = notcars[0:sample_size]

train_cars = cars
train_notcars = notcars

car_features = extract_features(train_cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(train_notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

output = open('svc_pickle.pkl', 'wb')
pickle.dump({"svc":svc,"X_scaler":X_scaler}, output)

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6108
5.4 Seconds to train SVC...
Test Accuracy of SVC =  0.989


# Explore Color Spaces

In [ ]:
for fname in glob.glob("examples/color_spaces/*.png"):
    # Read a color image
    img = cv2.imread(fname)

    # Select a small fraction of pixels to plot by subsampling it
    scale = max(img.shape[0], img.shape[1], 64) / 64  # at most 64 rows and columns
    img_small = cv2.resize(img, (np.int(img.shape[1] / scale), np.int(img.shape[0] / scale)), interpolation=cv2.INTER_NEAREST)

    # Convert subsampled image to desired color space(s)
    img_small_RGB = cv2.cvtColor(img_small, cv2.COLOR_BGR2RGB)  # OpenCV uses BGR, matplotlib likes RGB
    img_small_HSV = cv2.cvtColor(img_small, cv2.COLOR_BGR2HSV)
    img_small_HLS = cv2.cvtColor(img_small, cv2.COLOR_BGR2HLS)
    img_small_rgb = img_small_RGB / 255.  # scaled to [0, 1], only for plotting

    # Plot and show
    plot3d(img_small_RGB, img_small_rgb)
    plt.show()
    plot3d(img_small_HSV, img_small_rgb, axis_labels=list("HSV"))
    plt.show()
    plot3d(img_small_HLS, img_small_rgb, axis_labels=list("HLS"))
    plt.show()

# Spatial Binning of Color

In [ ]:
# Read in an image
# You can also read cutout2, 3, 4 etc. to see other examples
image = mpimg.imread('test_images/test1.jpg')
    
feature_vec = bin_spatial(image, size=(32, 32))

# Plot features
plt.plot(feature_vec)
plt.title('Spatially Binned Features')
plt.show()

# scikit-image HOG

In [19]:
# Read in cars and notcars
car_images = glob.glob('vehicles/*/*.png')
notcar_images = glob.glob('non-vehicles/*/*.png')
cars = []
notcars = []
for image in car_images:
    cars.append(image)
for image in notcar_images:
    notcars.append(image)
        
# Generate a random index to look at a car image
ind = np.random.randint(0, len(cars))
# Read in the image
image = mpimg.imread(cars[ind])
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
# Define HOG parameters
orient = 9
pix_per_cell = 8
cell_per_block = 2
# Call our function with vis=True to see an image output
features, hog_image = get_hog_features(gray, orient, 
                        pix_per_cell, cell_per_block, 
                        vis=True, feature_vec=False)


# Plot the examples
fig = plt.figure()
plt.subplot(121)
plt.imshow(image, cmap='gray')
plt.title('Example Car Image')
plt.subplot(122)
plt.imshow(hog_image, cmap='gray')
plt.title('HOG Visualization')
plt.show()

# Sliding Window Implementation

In [32]:
image = mpimg.imread('test_video_frames/test_video_25.jpg')

ystart = [400, 400, 400, 400]
ystop = [464, 496, 512, 628]
scale = [1.0, 1.5, 1.75, 2.0]
window_img = np.copy(image)
for search in zip(ystart, ystop, scale):

    windows = slide_window(window_img, x_start_stop=(0, image.shape[1]), y_start_stop=(search[0], search[1]), 
                        xy_window=(int(search[2]*64), int(search[2]*64)), xy_overlap=(0.5, 0.5))                       
    window_img = draw_boxes(window_img, windows, color=(0, 255, 255), thick=6)                    
plt.imshow(window_img)
plt.show()

# Hog Sub-sampling Window Search

In [34]:
img = mpimg.imread('test_images/test1.jpg')
bboxes = []
ystart = [400, 400, 400, 400]
ystop = [464, 496, 512, 628]
scale = [1.0, 1.5, 1.75, 2.0]

for search in zip(ystart, ystop, scale):
    temp_bboxes = find_cars(img, search[0], search[1], search[2], svc, X_scaler, color_space, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)
    bboxes += temp_bboxes
out_img = draw_boxes(img, bboxes)
plt.imshow(out_img)
plt.show()


# Multiple Detections and False-Positives

In [28]:
# Read in a pickle file with bboxes saved
# Each item in the "all_bboxes" list will contain a 
# list of boxes for one of the images shown above
box_list = bboxes

# Read in image similar to one shown above 
image = mpimg.imread('test_images/test5.jpg')
heat = np.zeros_like(image[:,:,0]).astype(np.float)

# Add heat to each box in box list
heat = add_heat(heat,box_list)
    
# Apply threshold to help remove false positives
heat = apply_threshold(heat,2)

# Visualize the heatmap when displaying    
heatmap = np.clip(heat, 0, 255)

# Find final boxes from heatmap using label function
labels = label(heatmap)
draw_img = draw_labeled_bboxes(np.copy(image), labels)

fig = plt.figure()
plt.subplot(121)
plt.imshow(draw_img)
plt.title('Car Positions')
plt.subplot(122)
plt.imshow(heatmap, cmap='hot')
plt.title('Heat Map')
fig.tight_layout()
plt.show()

TypeError: draw_labeled_bboxes() takes 1 positional argument but 2 were given

# Frame grabber by Tobi Lehman
https://tobilehman.com/blog/2013/01/20/extract-array-of-frames-from-mp4-using-python-opencv-bindings/

In [ ]:
fname = 'test_video.mp4'
vidcap = cv2.VideoCapture(fname)
#The easiest way to extract frames is to use the read() method on the vidcap object.
#It returns a tuple where the first element is a success flag and the second is the image array.
#To extract the image array:
success,image = vidcap.read()
# image is an array of array of [R,G,B] values
#To convert the whole video to frames and save each one:
count = 0; 
frames = 2000
while success:
    success,image = vidcap.read()
    cv2.imwrite("test_video_frames/{}_{}.jpg".format(fname[:-4], count), image)
    if count == frames:
        break
    count += 1